In [ ]:
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
from prep_data import *
import locale
# locale.setlocale(locale.LC_TIME, 'tr_TR.UTF8')
# 

# Sales

In [ ]:
df_totals_total, df_totals_cities, df_granular, df_granular_cities = sales_by_cities_df()

In [ ]:
df_totals_cities

In [ ]:
df_totals_cities.columns


In [ ]:
df_granular

In [ ]:

# def total_sales_plot(df_totals):

fig = go.Figure(data=[go.Line(name=city, x=df_totals_cities.index, y=df_totals_cities[city]) for city in df_totals_cities.columns], 
                layout=dict(height=800, width=1800))
fig.update_layout(title="İllere Göre Konut Satışı (Yıl Bazında)", xaxis_title="Yıl", yaxis_title="Toplam Konut Satışı", barmode='stack')
fig.update_xaxes(tickmode='linear')
fig.update_yaxes(tickmode='linear')

fig.show()

In [ ]:

# def total_sales_plot(df_totals):

fig = go.Figure(data=[go.Bar(name="Adana", x=df_totals_cities.index, y=df_totals_cities["1 Adana"])], 
                layout=dict(height=800, width=1800))
fig.update_layout(title="İllere Göre Konut Satışı (Yıl Bazında)", xaxis_title="Yıl", yaxis_title="Toplam Konut Satışı", barmode='stack')
fig.update_xaxes(tickmode='linear')
fig.update_yaxes(tickmode='linear')

fig.show()

In [ ]:
df_granular_cities = df_granular.drop("Total", axis=1)
df_granular_cities.head()

In [ ]:

# def total_sales_plot(df_totals):

fig = go.Figure(data=[go.Line(name=city, x=df_granular_cities.index, y=df_granular_cities[city]) for city in df_totals_cities.columns], 
                layout=dict(height=800, width=1800))
fig.update_layout(title="İllere Göre Konut Satışı (Ay bazında)", xaxis_title="Yıl", yaxis_title="Toplam Konut Satışı", barmode='stack')

fig.show()

In [ ]:
df_f = pd.read_excel("./datasets/İllere göre yabancılara konut satış sayısı.xls")
df_f.rename(columns={'Unnamed: 1': 'Şehir'}, inplace=True)
df_f.tail(14)

df_f_total = df_f[df_f['Şehir'] == "Toplam - Total"].drop(["Şehir", "Toplam"], axis=1)#.set_index("Yıl")
df_f_total["Yıl"] = df_f_total["Yıl"].astype(int)
# df_f_total.index = df_f_total.index.astype(int)

df_f["Yıl"] = df_f["Yıl"].ffill().astype(int)
df_f.drop("Toplam", axis=1, inplace=True)
df_f.tail(19)

In [ ]:
df_f.tail()

In [ ]:
df_f_total.tail()


In [ ]:
# Fix the 2024 data
# Reshape the DataFrame into long format
df_long = df_f_total.melt(id_vars=['Yıl'], var_name='Ay', value_name='Sayı')

# Aggregate the data by year, month, and city
df_f_total_aggregated = df_long.groupby(['Yıl', 'Ay']).sum().reset_index()

# Create a mapping of month names to their corresponding numbers
month_mapping = {
    'Ocak': 1, 'Şubat': 2, 'Mart': 3, 'Nisan': 4, 'Mayıs': 5, 'Haziran': 6,
    'Temmuz': 7, 'Ağustos': 8, 'Eylül': 9, 'Ekim': 10, 'Kasım': 11, 'Aralık': 12
}

# Convert 'Yıl' and 'Ay' to datetime format
df_f_total_aggregated['Tarih'] = df_f_total_aggregated.apply(lambda row: pd.Timestamp(int(row['Yıl']), month_mapping[row['Ay']], 1), axis=1)

# Drop the 'Yıl' and 'Ay' columns if not needed anymore
df_f_total_aggregated.drop(columns=['Yıl', 'Ay'], inplace=True)

df_f_total_aggregated = df_f_total_aggregated.sort_values(by='Tarih')
df_f_total_aggregated.set_index("Tarih", inplace=True)
df_f_total_aggregated["Sayı"] = df_f_total_aggregated["Sayı"].astype(int)
df_f_total_aggregated.info()

In [ ]:
df_f_total_aggregated.tail(15)

In [ ]:
df_f_cities = df_f[df_f['Şehir'] != "Toplam - Total"]
df_f_cities.head()

In [ ]:
# Fix the 2024 data
# Reshape the DataFrame into long format
df_long = df_f_cities.melt(id_vars=['Yıl', 'Şehir'], var_name='Ay', value_name='Sayı')

# Aggregate the data by year, month, and city
df_f_cities_aggregated = df_long.groupby(['Yıl', 'Şehir', 'Ay']).sum().reset_index()

# Create a mapping of month names to their corresponding numbers
month_mapping = {
    'Ocak': 1, 'Şubat': 2, 'Mart': 3, 'Nisan': 4, 'Mayıs': 5, 'Haziran': 6,
    'Temmuz': 7, 'Ağustos': 8, 'Eylül': 9, 'Ekim': 10, 'Kasım': 11, 'Aralık': 12
}

# Convert 'Yıl' and 'Ay' to datetime format
df_f_cities_aggregated['Tarih'] = df_f_cities_aggregated.apply(lambda row: pd.Timestamp(int(row['Yıl']), month_mapping[row['Ay']], 1), axis=1)

# Drop the 'Yıl' and 'Ay' columns if not needed anymore
df_f_cities_aggregated.drop(columns=['Yıl', 'Ay'], inplace=True)


df_f_cities_aggregated = df_f_cities_aggregated.sort_values(by='Tarih')
df_f_cities_aggregated.set_index("Tarih", inplace=True)
df_f_cities_aggregated["Sayı"] = df_f_cities_aggregated["Sayı"].astype(int)
df_f_cities_aggregated.info()

In [ ]:
df_f_cities_aggregated.head(20)

In [ ]:

def sales_by_cities_foreigners_df(start_year=2013, end_year=2024):
    """
        This function returns the number of real estates sold to foreigners in each city (İl)

        params:
            start_year (int): the starting year for the total number of sales in the country
            end_year (int): the ending year for the total number of sales in the country

        returns:
            df_totals_total (pandas_df): Total number of real estates sold in the country
            df_totals_cities (pandas_df): Total number of real estates sold in each city
            df_granular (pandas_df): Number of real estates sold in the country in total a between [start_year, end_year] and in each city
                in each month (monthly granularity)
            df_granular_cities (pandas_df): df_granular without the "Total" column 
    """

    df_f = pd.read_excel("./datasets/İllere göre yabancılara konut satış sayısı.xls")
    df_f.rename(columns={'Unnamed: 1': 'Şehir'}, inplace=True)
    df_f_total = df_f[df_f['Şehir'] == "Toplam - Total"].drop(["Şehir", "Toplam"], axis=1)#.set_index("Yıl")
    df_f_total["Yıl"] = df_f_total["Yıl"].astype(int)
    df_f["Yıl"] = df_f["Yıl"].ffill().astype(int)
    df_f.drop("Toplam", axis=1, inplace=True)

    df_long = df_f_total.melt(id_vars=['Yıl'], var_name='Ay', value_name='Total')
    df_f_total_aggregated = df_long.groupby(['Yıl', 'Ay']).sum().reset_index()
    month_mapping = {
        'Ocak': 1, 'Şubat': 2, 'Mart': 3, 'Nisan': 4, 'Mayıs': 5, 'Haziran': 6,
        'Temmuz': 7, 'Ağustos': 8, 'Eylül': 9, 'Ekim': 10, 'Kasım': 11, 'Aralık': 12
    }
    df_f_total_aggregated['Tarih'] = df_f_total_aggregated.apply(lambda row: pd.Timestamp(int(row['Yıl']), month_mapping[row['Ay']], 1), axis=1)
    df_f_total_aggregated.drop(columns=['Yıl', 'Ay'], inplace=True)
    df_f_total_aggregated = df_f_total_aggregated.sort_values(by='Tarih')
    df_f_total_aggregated.set_index("Tarih", inplace=True)
    df_f_total_aggregated["Total"] = df_f_total_aggregated["Total"].astype(int)

    df_f_cities = df_f[df_f['Şehir'] != "Toplam - Total"]
    df_long = df_f_cities.melt(id_vars=['Yıl', 'Şehir'], var_name='Ay', value_name='Total')
    df_f_cities_aggregated = df_long.groupby(['Yıl', 'Şehir', 'Ay']).sum().reset_index()
    df_f_cities_aggregated['Tarih'] = df_f_cities_aggregated.apply(lambda row: pd.Timestamp(int(row['Yıl']), month_mapping[row['Ay']], 1), axis=1)
    df_f_cities_aggregated.drop(columns=['Yıl', 'Ay'], inplace=True)
    df_f_cities_aggregated = df_f_cities_aggregated.sort_values(by='Tarih')
    df_f_cities_aggregated.set_index("Tarih", inplace=True)
    df_f_cities_aggregated["Total"] = df_f_cities_aggregated["Total"].astype(int)

    df_f_total_aggregated = df_f_total_aggregated[(df_f_total_aggregated.index.year <= end_year) & (df_f_total_aggregated.index.year >= start_year)]
    df_f_total_aggregated = df_f_total_aggregated[df_f_total_aggregated['Total'] != 0]

    df_f_cities_aggregated = df_f_cities_aggregated[(df_f_cities_aggregated.index.year <= end_year) & (df_f_cities_aggregated.index.year >= start_year)]
    df_f_cities_aggregated = df_f_cities_aggregated[df_f_cities_aggregated['Total'] != 0]


    return df_f_total_aggregated, df_f_cities_aggregated

df_f_total_aggregated, df_f_cities_aggregated = sales_by_cities_foreigners_df()

# Population

In [ ]:
df_p = pd.read_excel("./datasets/favori_raporlar.xlsx", header=[0, 1])

# df_p.columns = ['-'.join(col).strip().lower() for col in df_p.columns.values]
# df_p_new_cols = df_p.columns.to_list()
# for i in range(0,3):
#     df_p_new_cols[i] = df_p_new_cols[i].split("-")[0]
# df_p.columns = df_p_new_cols

cols = ['il kayit no', 'il adi', 'toplam', 'toplam-erkek', 'toplam-kadin',
       'il ve ilçe merkezleri-toplam', 'il ve ilçe merkezleri-erkek',
       'il ve ilçe merkezleri-kadin', 'belde ve köyler-toplam',
       'belde ve köyler-erkek', 'belde ve köyler-kadin']
# df_p.columns = ['-'.join(col).strip().lower() if i >= 3 else col[0].strip().lower() for i, col in enumerate(df_p.columns.values)]
df_p.columns = cols

df_p.head()

In [ ]:
# df_p = df_p[0:1].drop(columns=[df_p.columns[0], df_p.columns[1]])
# df_p
df_p.iloc[0,0] = 0
df_p.iloc[0,1] = "ÜLKE"
df_p.drop("il kayit no", axis=1,inplace=True)
df_p

In [ ]:
df_p.info()

In [ ]:
df_p_cities = df_p[[df_p.columns[0], df_p.columns[2], df_p.columns[3]]]
df_p_cities = df_p_cities[1:]
df_p_cities


In [ ]:
df_p[["il adi", "toplam-erkek"]]

In [ ]:
# Pie chart
# choloropleth

In [ ]:
import plotly.graph_objects as go

# Sample data for the pie chart
labels = ['Erkek', 'Kadın']
values = [df_p.iloc[0, 1], df_p.iloc[0, 2]]
colors = ['skyblue', 'salmon']

# Create the Pie chart with custom colors and slice pulled out
fig = go.Figure(go.Pie(
    labels=labels,
    values=values,
    marker=dict(colors=colors),
    hovertemplate='<b>%{label}</b><br>%{value}'
), layout=dict(width=400, height=400))

# Update the name of the trace
fig.data[0].name = ''

# Set the title
fig.update_layout(title='Cinsiyet Dağılımı')

# Show the figure
fig.show()


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
def population_plot(df_p, city_code=0):
    # Create the figure with subplots
    fig = make_subplots(rows=1, cols=3, specs=[[{'type': 'pie'}, {'type': 'pie'}, {'type': 'pie'}]], subplot_titles=(df_p.iloc[city_code, 0].capitalize(), "İl ve İlçe Merkezleri", "Belde ve Köyler"))

    # Define the data for each pie chart
    values1 = [df_p.iloc[city_code, 2], df_p.iloc[city_code, 3]]
    values2 = [df_p.iloc[city_code, 5], df_p.iloc[city_code, 6]]
    values3 = [df_p.iloc[city_code, 8], df_p.iloc[city_code, 9]]
    labels = ['Erkek', 'Kadın']
    colors = ['skyblue', 'salmon']

    # Add the first pie chart
    fig.add_trace(go.Pie(
        name="",
        labels=labels,
        values=values1,
        marker=dict(colors=colors),
        hovertemplate='<b>%{label}</b><br>%{value}',
        pull=[0, 0],
        textfont=dict(size=16, family="Balto", color="black"),  # Adjust text size and color
        textinfo='label+percent',  # Show labels and percentages
    ), row=1, col=1)

    # Add the second pie chart
    fig.add_trace(go.Pie(
        name="",
        labels=labels,
        values=values2,
        marker=dict(colors=colors),
        hovertemplate='<b>%{label}</b><br>%{value}',
        pull=[0, 0],
        textfont=dict(size=16, family="Balto", color="black"),  # Adjust text size and color
        textinfo='label+percent',  # Show labels and percentages
    ), row=1, col=2)

    # Add the third pie chart
    fig.add_trace(go.Pie(
        name="",
        labels=labels,
        values=values3,
        marker=dict(colors=colors),
        hovertemplate='<b>%{label}</b><br>%{value}',
        pull=[0, 0],
        textfont=dict(size=16, family="Balto", color="black"),  # Adjust text size and color
        textinfo='label+percent',  # Show labels and percentages
    ), row=1, col=3)

    # Update layout
    fig.update_layout(
        title='Cinsiyet Dağılımları',
        width=1100,
        height=500,
        title_font=dict(size=30, family="Balto", ),  # Adjust title font
        title_pad_b=10,
        font=dict(
            family="Balto",  # Adjust default font family for labels
            size=14,  # Adjust default font size for labels
            color="black"  # Adjust default font color for labels
        ),
        legend=dict(
            title_font=dict(size=18, family="Balto"),  # Adjust legend title font
            font=dict(size=16, family="Balto")  # Adjust legend item font
        )
    )

    # Update subplot titles font and style

    for annotation in fig['layout']['annotations']:
        annotation['y'] = 0.95  # Adjust this value for more/less padding

        # Add the text annotation below the plots
    fig.add_annotation(
    text=f"Toplam Nüfus: <b>{df_p.iloc[city_code, 1]}</b>",
    xref="paper", yref="paper",
    x=0.5, y=-0.2,
    showarrow=False,
    font=dict(size=18, family="Balto", color="black"),
    align="center"
)
    return fig
fig = population_plot(df_p=df_p, city_code=1)
# Show the figure
fig.show()


In [ ]:
import plotly.express as px

# Assuming you have the GeoJSON file for Turkey provinces
geojson_file = './tr-cities.json'

# Calculate total population
df_p_cities['total_population'] = df_p_cities['toplam-erkek'] + df_p_cities['toplam-kadin']

# Load GeoJSON data
import json
with open(geojson_file) as f:
    turkey_geojson = json.load(f)

# Ensure the province names in the DataFrame match the names in the GeoJSON file
# This might require some preprocessing if the names don't match exactly


# Create the choropleth map
fig = px.choropleth(
    df_p_cities,
    geojson=turkey_geojson,
    locations='il adi',
    featureidkey="properties.name",  # This should match the property name in your GeoJSON file for province names
    color='total_population',
    hover_name='il adi',
    hover_data={'toplam-erkek': True, 'toplam-kadin': True, 'total_population': True},
    color_continuous_scale='Viridis',
    labels={'total_population': 'Total Population'},
    title='Total Population by Province in Turkey'
)

# Update the layout for better visualization
fig.update_geos(
    visible=False,
    resolution=50,
    showcountries=True,
    countrycolor="Black",
    showcoastlines=True,
    coastlinecolor="Black",
    showland=True,
    landcolor="white",
)

fig.update_layout(
    title_font=dict(size=24, family="Arial, sans-serif"),
    font=dict(size=12, family="Arial, sans-serif"),
    margin={"r":0,"t":50,"l":0,"b":0}
)

# Show the map
# fig.show()


In [ ]:
turkey_geojson.keys()

In [ ]:
turkey_geojson["features"][2]["properties"]["name"]

In [ ]:
def total_sales_monthly_foreigners_plot(df_f_cities_aggregated, city=None):
    """
        City must be spelled properly
    """
    if city == None:
        fig = px.line(df_f_cities_aggregated, x=df_f_cities_aggregated.index, 
                      y='Total', color='Şehir', title='Real Estate Sold Over Time',
                      )
    else:
        city = city.capitalize()
        cities = df_f_cities_aggregated["Şehir"].unique()
        if city not in cities:
            city = 'Diğer iller - Other provinces'
        dff = df_f_cities_aggregated[df_f_cities_aggregated['Şehir'] == city]
        fig = px.line(dff, x=dff.index, y='Total', color='Şehir', title='Real Estate Sold Over Time',
                     color_discrete_sequence=px.colors.qualitative.Vivid)
    fig.update_layout(width=1800, height=800)
    return fig

fig = total_sales_monthly_foreigners_plot(df_f_cities_aggregated, "123")
fig.show()